<a href="https://colab.research.google.com/github/tarunku/open_llm/blob/main/LangChain_Runnable_Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain-openai langchain_community

# **RunnableLambda** is a class in LangChain

that allows you to wrap a Python function as a runnable component in a chain. It essentially turns your function into a building block that can be connected with other components to create complex workflows.


**Why it's used:**

Flexibility: It enables you to integrate custom logic and functions into your LangChain chains.
Data manipulation: You can use it to pre-process or post-process data within the chain.
Dynamic behavior: It allows you to create chains that adapt to different inputs or conditions.


# **RunnableSequence** a class in LangChain
that allows you to define a sequence of runnable components that will be executed in order. It's like creating a pipeline where the output of one component becomes the input of the next.

Why it's used:

Workflow orchestration: It helps you define and manage complex workflows involving multiple steps.
Modular design: It promotes code reusability by allowing you to combine and reuse individual components.
Control flow: It gives you control over the order of execution and data flow between components.


# To creates a sequence:

*   step1 (the RunnableLambda wrapping get_random_country) is executed first.
*   step2 (asking for the popular city) is executed next, using the output of step1 as input.
*   step3 (asking for things to do) is executed last, using the output of step2 as input.

By using RunnableLambda and RunnableSequence, code becomes more modular, flexible, and easier to understand and maintain.

In [ ]:
import random
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from google.colab import userdata


def get_random_country(country=None):
  """Returns a random country name from the list.

  Args:
    countries: A list of country names.

  Returns:
    A random country name from the list.
  """
  country_list = ["United States", "Canada", "Mexico", "Brazil", "India", "China", "Japan"]
  return random.choice(country_list)

template_1 = "What is the most popular city in {country} for tourists? Just return the name of the city"
template_2 = "What are the top three things to do in this: {city} for tourists. Just return the answer as three bullet points."

prompt_1 = ChatPromptTemplate.from_template(template_1)
prompt_2 = ChatPromptTemplate.from_template(template_2)

output_parser = StrOutputParser()

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get('OPENAI_API_KEY'))


#runnable_sequence = RunnableLambda(get_random_country) | prompt_1 | model | output_parser | prompt_2 | model | output_parser

# Define the individual steps as Runnable objects
step1 = RunnableLambda(get_random_country)
step2 = prompt_1 | model | output_parser  # Combine prompt, model, and parser
step3 = prompt_2 | model | output_parser  # Combine prompt, model, and parser

#runnable_sequence = step1 | step2 | step3

# Initialize RunnableSequence
runnable_sequence = RunnableSequence(
    first=step1,
    middle=[step2],  # Put step2 in a list for the "middle" argument
    last=step3
)

runnable_sequence.invoke(None)




